# 4. CDC Latency - Oracle → Aurora

Queries Aurora PostgreSQL to compute CDC replication latency: `_sink_ts_ms - _source_ts_ms` (milliseconds from Oracle commit to Aurora write).

**Requires:** SMTs in 7-debezium-sink that add `_source_ts_ms` and `_sink_ts_ms` columns. Run from SageMaker (inside VPC) to reach Aurora.

In [ ]:
! pip install psycopg2-binary boto3 -q

In [ ]:
import boto3
import psycopg2

def run_sql(conn, sql):
    """Execute SQL and return DataFrame using psycopg2."""
    with conn.cursor() as cur:
        cur.execute(sql)
        if cur.description:
            cols = [d[0] for d in cur.description]
            rows = cur.fetchall()
            return pd.DataFrame(rows, columns=cols)
    return pd.DataFrame()

In [ ]:
ssm = boto3.client('ssm')
aurora_host = ssm.get_parameter(Name='/aurora/host', WithDecryption=True)['Parameter']['Value']
aurora_port = ssm.get_parameter(Name='/aurora/port', WithDecryption=True)['Parameter']['Value']
aurora_user = ssm.get_parameter(Name='/aurora/user', WithDecryption=True)['Parameter']['Value']
aurora_pass = ssm.get_parameter(Name='/aurora/password', WithDecryption=True)['Parameter']['Value']
aurora_db   = ssm.get_parameter(Name='/aurora/database', WithDecryption=True)['Parameter']['Value']

conn = psycopg2.connect(
    host=aurora_host,
    port=int(aurora_port),
    user=aurora_user,
    password=aurora_pass,
    dbname=aurora_db
)

In [ ]:
# Optional: list CDC table columns (table name may vary: oracle_cdc_admin_tags or oracle_cdc_ADMIN_TAGS)
try:
    check = run_sql(conn, """
        SELECT column_name FROM information_schema.columns 
        WHERE table_schema='public' AND table_name LIKE 'oracle_cdc%' 
        ORDER BY ordinal_position
    """)
    print("CDC table columns:", check['column_name'].tolist())
except Exception as e:
    print(f"Note: {e}")

## Latency query

`_source_ts_ms`: timestamp when change was committed in Oracle (from Debezium)
`_sink_ts_ms`: timestamp when JDBC Sink wrote to Aurora
`latency_ms`: end-to-end replication latency

In [ ]:
# Table: topic oracle-cdc.ADMIN.TAGS -> oracle_cdc_admin_tags (PostgreSQL lowercase)
# If table not found, try: oracle_cdc_ADMIN_TAGS
TABLE_NAME = "oracle_cdc_admin_tags"
query = f"""
SELECT 
    _source_ts_ms,
    _sink_ts_ms,
    (_sink_ts_ms - _source_ts_ms) AS latency_ms
FROM {TABLE_NAME}
WHERE _source_ts_ms IS NOT NULL AND _sink_ts_ms IS NOT NULL
ORDER BY _sink_ts_ms DESC
LIMIT 100;
"""

df = run_sql(conn, query)
df

## Latency statistics

In [ ]:
stats_query = f"""
SELECT 
    COUNT(*) AS total_records,
    ROUND(AVG(_sink_ts_ms - _source_ts_ms)::numeric, 2) AS avg_latency_ms,
    MIN(_sink_ts_ms - _source_ts_ms) AS min_latency_ms,
    MAX(_sink_ts_ms - _source_ts_ms) AS max_latency_ms,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY (_sink_ts_ms - _source_ts_ms)) AS p50_latency_ms,
    PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY (_sink_ts_ms - _source_ts_ms)) AS p95_latency_ms,
    PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY (_sink_ts_ms - _source_ts_ms)) AS p99_latency_ms
FROM {TABLE_NAME}
WHERE _source_ts_ms IS NOT NULL AND _sink_ts_ms IS NOT NULL;
"""

stats = run_sql(conn, stats_query)
stats.T

In [ ]:
conn.close()